In [7]:
import openmm.app as app
import openmm as mm
import openmm.unit as unit
import numpy as np
import jax
import jax.numpy as jnp
import dmff
from dmff.api.xmlio import XMLIO
from dmff.api.paramset import ParamSet
from dmff.generators.classical import CoulombGenerator, LennardJonesGenerator
from dmff.api.hamiltonian import Hamiltonian
from dmff.operators import ParmedLennardJonesOperator
from dmff import NeighborListFreud
from dmff.mbar import ReweightEstimator
import mdtraj as md
from tqdm import tqdm, trange
import parmed
import sys
import os
from dmff.api.topology import DMFFTopology

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.


Here I construct the initial force field Lenard-Jone potential and Coulume potential from UFF and MIL-120 from aiida-plugin

The force field file in RASPA: epsilon units (Kelvin); sigma(Angstrom)

Al_     LENNARD_JONES   254.152 4.0082
C_      LENNARD_JONES   52.8435 3.4309
H_      LENNARD_JONES   22.1439 2.5711
O_      LENNARD_JONES   30.1963 3.1181
O_co2   LENNARD_JONES   79.0    3.05
C_co2   LENNARD_JONES   27.0    2.80

The force field file in OpenMM: epsilon units (KJ/mol); sigma(nm)

Al_     LENNARD_JONES   2.1019    0.40082
C_      LENNARD_JONES   0.4400    0.34309
H_      LENNARD_JONES   0.1841    0.25711
O_      LENNARD_JONES   0.2513    0.31181
O_co2   LENNARD_JONES   0.6577    0.30500
C_co2   LENNARD_JONES   0.2249    0.28000



In [69]:
# masses for different elements
elem_masses = {
    'Al': 26.9815386,  # Mass of Aluminum (Al) in amu
    'O': 15.999,       # Mass of Oxygen (O) in amu
    'C': 12.011,       # Mass of Carbon (C) in amu
    'H': 1.00784       # Mass of Hydrogen (H) in amu
}

# get Lenard-Jones potential parameters according to different element from def file

lj_parameters = {
    'Al_': {
        'epsilon': 2.1019,  # kJ/mol
        'sigma': 0.40082  # nm
    },
    'C_': {
        'epsilon': 0.4400,  # kJ/mol
        'sigma': 0.34309  # nm
    },
    'H_': {
        'epsilon': 0.1841,  # kJ/mol
        'sigma': 0.25711  # nm
    },
    'O_': {
        'epsilon': 0.2513,  # kJ/mol
        'sigma': 0.31181  # nm
    },
    'O_co2': {
        'epsilon': 0.6577,  # kJ/mol
        'sigma': 0.30500  # nm
    },
    'C_co2': {
        'epsilon': 0.2249,  # kJ/mol
        'sigma': 0.28000  # nm
    }
}

# get atomic information from cif files

# firstly, I construct the system for the MIL-120 framework

from CifFile import ReadCif

# Load the CIF file
cif_file = "/home/yutao/softwares/RASPA2/../MIL-120/MIL-120.cif"

# Read the CIF file
cif_data = ReadCif(cif_file)

# Extract information for each atom
atom_data = []

# Loop through the CIF data blocks
for block in cif_data:
    if '_atom_site_label' in block and '_atom_site_charge' in block and '_atom_site_fract_x' in block and '_atom_site_fract_y' in block and '_atom_site_fract_z' in block:
        labels = block['_atom_site_label']
        charge_values = block['_atom_site_charge']
        x_positions = block['_atom_site_fract_x']
        y_positions = block['_atom_site_fract_y']
        z_positions = block['_atom_site_fract_z']

        for label, charge, x, y, z in zip(labels, charge_values, x_positions, y_positions, z_positions):
            atom_data.append({
                'label': label,
                'charge': float(charge),
                'position': np.array([float(x), float(y), float(z)]),
                'epsilon': lj_parameters[label+'_']['epsilon'],
                'sigma': lj_parameters[label+'_']['sigma']
            })

# Now you have a list of dictionaries, each containing label, charge, and position
print(atom_data)

[{'label': 'Al', 'charge': 1.65857, 'position': array([0.29166, 0.29186, 0.58401]), 'epsilon': 2.1019, 'sigma': 0.40082}, {'label': 'Al', 'charge': 1.65856, 'position': array([0.70833, 0.70814, 0.416  ]), 'epsilon': 2.1019, 'sigma': 0.40082}, {'label': 'Al', 'charge': 1.64626, 'position': array([-0. ,  0.5,  0.5]), 'epsilon': 2.1019, 'sigma': 0.40082}, {'label': 'Al', 'charge': 1.64749, 'position': array([ 0.5, -0. ,  0.5]), 'epsilon': 2.1019, 'sigma': 0.40082}, {'label': 'H', 'charge': 0.41785, 'position': array([0.01775, 0.34267, 0.67087]), 'epsilon': 0.1841, 'sigma': 0.25711}, {'label': 'H', 'charge': 0.41373, 'position': array([0.14093, 0.93835, 0.4632 ]), 'epsilon': 0.1841, 'sigma': 0.25711}, {'label': 'H', 'charge': 0.41657, 'position': array([0.65156, 0.32865, 0.67108]), 'epsilon': 0.1841, 'sigma': 0.25711}, {'label': 'H', 'charge': 0.41461, 'position': array([0.32343, 0.52517, 0.46387]), 'epsilon': 0.1841, 'sigma': 0.25711}, {'label': 'H', 'charge': 0.41773, 'position': array([

In [4]:
from openbabel import openbabel as ob
# Create an Open Babel molecule object

mol = ob.OBMol()

# Read the input MOL file
input_pdbfile = "MIL120_loading.pdb"
input_format = "pdb"
obConversion = ob.OBConversion()
obConversion.SetInAndOutFormats(input_format, input_format)
obConversion.ReadFile(mol, input_pdbfile)

# Remove CONECT lines
for atom in ob.OBMolAtomIter(mol):
    atom.DeleteData("connect")

# Write the modified molecule to a new PDB file
output_pdbfile = "MIL120_loading.mol2"
output_format = "mol2"
obConversion.SetInAndOutFormats(output_format, output_format)
obConversion.WriteFile(mol, output_pdbfile)


*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 79-80 should contain charge of the atom
  but OpenBabel found '_c' (atom 41).
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 79-80 should contain charge of the atom
  but OpenBabel found '_c' (atom 42).
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 79-80 should contain charge of the atom
  but OpenBabel found '_c' (atom 43).


True

In [3]:
output_pdbfile = "MIL-120-new.gro"
output_format = "gro"
obConversion.SetInAndOutFormats(output_format, output_format)
obConversion.WriteFile(mol, output_pdbfile)

True

In [2]:
import openbabel
import subprocess

import shutil

# Step 1: Convert MOL2 to GRO

obConversion = openbabel.OBConversion()
obConversion.SetInAndOutFormats("mol2", "gro")

mol = openbabel.OBMol()
obConversion.ReadFile(mol, "MIL-120-new.mol2")
obConversion.WriteFile(mol, "MIL-120-new.gro")

: 

In [ ]:
from openbabel import openbabel as ob
# Create an Open Babel molecule object
mol = ob.OBMol()

# Read the input MOL file
input_pdbfile = "Framework_final.pdb"
input_format = "pdb"
obConversion = ob.OBConversion()
obConversion.SetInAndOutFormats(input_format, input_format)
obConversion.ReadFile(mol, input_pdbfile)

# Remove CONECT lines
for atom in ob.OBMolAtomIter(mol):
    atom.DeleteData("connect")

# Write the modified molecule to a new PDB file
output_pdbfile = "MIL-120-new.pdb"
output_format = "pdb"
obConversion.SetInAndOutFormats(output_format, output_format)
obConversion.WriteFile(mol, output_pdbfile)


In [64]:
"""

Construct Lenard-Jones potential part for MIL-120 structure.

"""



import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom

# Create the root element
root = ET.Element("ForceField")

# Create the LennardJonesForce element
lj_force = ET.SubElement(root, "LennardJonesForce")
lj_force.set("lj14scale", "0.50000")


# Add Atom elements for each atom
for symbol in lj_parameters.keys():
    atom = ET.SubElement(lj_force, "Atom")
    atom.set("epsilon", str(lj_parameters[symbol]["epsilon"]))
    atom.set("sigma", str(lj_parameters[symbol]["sigma"]))
    atom.set("type", symbol)

# Create an ElementTree
tree = ET.ElementTree(root)

# Convert the ElementTree to a formatted string with line breaks
xml_string = minidom.parseString(ET.tostring(root)).toprettyxml(indent="    ")

# Remove the XML declaration line
xml_lines = xml_string.split("\n")[1:]

# Write the formatted XML string to a file
with open("custom_forcefield_new.xml", "w") as xml_file:
    xml_file.write("\n".join(xml_lines))

print("XML file created: custom_forcefield.xml")


XML file created: custom_forcefield.xml


In [ ]:
particle_number =  500             #液体pdb文件中的分子数
target_han= 50.52                  #拟合的目标蒸发焓
SET_temperature=  293.15           #温度设定
time_gap=   2                      #分子动力学模拟过程中每一个frame的时间间隔，单位是皮秒picosecond   推荐2-4ps
loop_time =   50                   #迭代循环次数    推荐50-100
save_step =   125                  #保存的frame数  推荐150-200
skip_step =    25                   #计算各种物理常数之前，达到平衡所需要的步数
length_step = (skip_step+save_step) #分子动力学模拟生成的dcd文件存储的frame个数，与time_gap共同决定总模拟的时长，总时长=length_step*time_gap

In [113]:
pdb = app.PDBFile("MIL120_loading.pdb")

# Add print statements for debugging
for atom in pdb.topology.atoms():
    continue
    print(f"Atom: {atom.name}, Residue: {atom.residue.name}, Atom Type: {atom.element}")
ff = app.ForceField("custom_forcefield_new.xml")
system = ff.createSystem(pdb.topology, nonbondedMethod=app.Ewald, constraints=None, removeCMMotion=False)


In [109]:
pdb = app.PDBFile("MIL120_loading.pdb")

# Add print statements for debugging
for atom in pdb.topology.atoms():
    print(f"Atom: {atom.name}, Residue: {atom.residue.name}, Atom Type: {atom.element}")

Atom: Al1, Residue: MOL, Atom Type: <Element aluminum>
Atom: Al2, Residue: MOL, Atom Type: <Element aluminum>
Atom: Al3, Residue: MOL, Atom Type: <Element aluminum>
Atom: Al4, Residue: MOL, Atom Type: <Element aluminum>
Atom: H1, Residue: MOL, Atom Type: <Element hydrogen>
Atom: H2, Residue: MOL, Atom Type: <Element hydrogen>
Atom: H3, Residue: MOL, Atom Type: <Element hydrogen>
Atom: H4, Residue: MOL, Atom Type: <Element hydrogen>
Atom: H5, Residue: MOL, Atom Type: <Element hydrogen>
Atom: H6, Residue: MOL, Atom Type: <Element hydrogen>
Atom: H7, Residue: MOL, Atom Type: <Element hydrogen>
Atom: H8, Residue: MOL, Atom Type: <Element hydrogen>
Atom: H9, Residue: MOL, Atom Type: <Element hydrogen>
Atom: H10, Residue: MOL, Atom Type: <Element hydrogen>
Atom: C1, Residue: MOL, Atom Type: <Element carbon>
Atom: C2, Residue: MOL, Atom Type: <Element carbon>
Atom: C3, Residue: MOL, Atom Type: <Element carbon>
Atom: C4, Residue: MOL, Atom Type: <Element carbon>
Atom: C5, Residue: MOL, Atom Ty

In [72]:
for force_index in range(system.getNumForces()):
    force = system.getForce(force_index)
    if isinstance(force, openmm.NonbondedForce):
        for particle_index in range(force.getNumParticles()):
            charge, sigma, epsilon = force.getParticleParameters(particle_index)
            print(f'Particle {particle_index}: Charge={charge}, Sigma={sigma}, Epsilon={epsilon}')

<class 'openmm.app.forcefield.ForceField'>


In [115]:
# Assuming you have a CustomNonbondedForce in the system
for force_index in range(system.getNumForces()):
    force = system.getForce(force_index)
    if isinstance(force, mm.NonbondedForce):
        for particle_index in range(force.getNumParticles()):
            parameters = force.getParticleParameters(particle_index)
            print(parameters)
            mass = system.getParticleMass(particle_index)
            charge = parameters[0]  # Assuming charge is the first parameter
            print(f"Particle {particle_index + 1}: Mass = {mass}, Charge = {charge}")

[Quantity(value=1.65857, unit=elementary charge), Quantity(value=0.40082, unit=nanometer), Quantity(value=2.1019, unit=kilojoule/mole)]
Particle 1: Mass = 0.0 Da, Charge = 1.65857 e
[Quantity(value=1.65856, unit=elementary charge), Quantity(value=0.40082, unit=nanometer), Quantity(value=2.1019, unit=kilojoule/mole)]
Particle 2: Mass = 0.0 Da, Charge = 1.65856 e
[Quantity(value=1.64626, unit=elementary charge), Quantity(value=0.40082, unit=nanometer), Quantity(value=2.1019, unit=kilojoule/mole)]
Particle 3: Mass = 0.0 Da, Charge = 1.64626 e
[Quantity(value=1.64749, unit=elementary charge), Quantity(value=0.40082, unit=nanometer), Quantity(value=2.1019, unit=kilojoule/mole)]
Particle 4: Mass = 0.0 Da, Charge = 1.64749 e
[Quantity(value=0.41785, unit=elementary charge), Quantity(value=0.25711, unit=nanometer), Quantity(value=0.1841, unit=kilojoule/mole)]
Particle 5: Mass = 0.0 Da, Charge = 0.41785 e
[Quantity(value=0.41373, unit=elementary charge), Quantity(value=0.25711, unit=nanometer),

In [104]:
# Assuming you have a CustomNonbondedForce in the system
for force_index in range(system.getNumForces()):
    force = system.getForce(force_index)
    if isinstance(force, mm.CustomNonbondedForce):
        for particle_index in range(force.getNumParticles()):
            force.setParticleParameters(particle_index, (5,1,1))
            parameters = force.getParticleParameters(particle_index)
            print(parameters)
            mass = system.getParticleMass(particle_index)
            charge = parameters[0]  # Assuming charge is the first parameter
            print(f"Particle {particle_index + 1}: Mass = {mass}, Charge = {charge}")


(5.0, 1.0, 1.0)
Particle 1: Mass = 0.0 Da, Charge = 5.0
(5.0, 1.0, 1.0)
Particle 2: Mass = 0.0 Da, Charge = 5.0
(5.0, 1.0, 1.0)
Particle 3: Mass = 0.0 Da, Charge = 5.0
(5.0, 1.0, 1.0)
Particle 4: Mass = 0.0 Da, Charge = 5.0
(5.0, 1.0, 1.0)
Particle 5: Mass = 0.0 Da, Charge = 5.0
(5.0, 1.0, 1.0)
Particle 6: Mass = 0.0 Da, Charge = 5.0
(5.0, 1.0, 1.0)
Particle 7: Mass = 0.0 Da, Charge = 5.0
(5.0, 1.0, 1.0)
Particle 8: Mass = 0.0 Da, Charge = 5.0
(5.0, 1.0, 1.0)
Particle 9: Mass = 0.0 Da, Charge = 5.0
(5.0, 1.0, 1.0)
Particle 10: Mass = 0.0 Da, Charge = 5.0
(5.0, 1.0, 1.0)
Particle 11: Mass = 0.0 Da, Charge = 5.0
(5.0, 1.0, 1.0)
Particle 12: Mass = 0.0 Da, Charge = 5.0
(5.0, 1.0, 1.0)
Particle 13: Mass = 0.0 Da, Charge = 5.0
(5.0, 1.0, 1.0)
Particle 14: Mass = 0.0 Da, Charge = 5.0
(5.0, 1.0, 1.0)
Particle 15: Mass = 0.0 Da, Charge = 5.0
(5.0, 1.0, 1.0)
Particle 16: Mass = 0.0 Da, Charge = 5.0
(5.0, 1.0, 1.0)
Particle 17: Mass = 0.0 Da, Charge = 5.0
(5.0, 1.0, 1.0)
Particle 18: Mass = 0.0 

In [65]:

# Load the PDB file
pdb = app.PDBFile("MIL120_loading_new.pdb")

# Add print statements for debugging
ff = app.ForceField("custom_forcefield.xml")
# Load the force field from the XML file
forcefield = app.ForceField("custom_forcefield.xml")

# Create the system
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.Ewald, constraints=None, removeCMMotion=False)


In [70]:
import openmm
# Load the PDB file
pdb = app.PDBFile("MIL120_loading_new.pdb")

# Load the force field from the XML file
forcefield = app.ForceField("custom_forcefield.xml")

# Create the system
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.Ewald, constraints=None, removeCMMotion=False)

# Manually set charges and masses
for i, atom in enumerate(pdb.topology.atoms()):
    # Get the atom name and residue name
    atom_name = atom.name
    residue_name = atom.residue.name

    # Get the charge and mass from the force field
    charge = 1 # Assuming charge is the first parameter
    mass = 12

    # Manually set charge and mass in the OpenMM system
    system.setParticleMass(i, mass)
    system.setParticleParameters(i, charge, 0.0, 0.0)  # Assuming 0.0 for the sigma and epsilon parameters

# Now the system should have the correct charges and masses


AttributeError: 'System' object has no attribute 'setParticleParameters'

In [69]:
forcefield.getParticleParameters

AttributeError: 'ForceField' object has no attribute 'getParticleParameters'

In [54]:
force.getNumComputedValues

<openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x7f2471ff2430> >

In [52]:
# Assuming you have a NonbondedForce in the system
for force_index in range(system.getNumForces()):
    force = system.getForce(force_index)
    if isinstance(force, mm.NonbondedForce):
        for particle_index in range(force.getNumParticles()):
            charge, sigma, epsilon = force.getParticleParameters(particle_index)
            mass = system.getParticleMass(particle_index)
            print(f"Particle {particle_index + 1}: Mass = {mass}, Charge = {charge}")


In [41]:
import simtk.openmm.app as app

# Load PDB file
pdb = app.PDBFile("MIL120_loading_new.pdb")

# Print atom and residue information
for atom in pdb.topology.atoms():
    print(f"Atom: {atom.name}, Residue: {atom.residue.name}, Residue Index: {atom.residue.index} Atom: {atom.element.symbol}")


Atom: Al1, Residue: MOL, Residue Index: 0 Atom: Al
Atom: Al2, Residue: MOL, Residue Index: 0 Atom: Al
Atom: Al3, Residue: MOL, Residue Index: 0 Atom: Al
Atom: Al4, Residue: MOL, Residue Index: 0 Atom: Al
Atom: H1, Residue: MOL, Residue Index: 0 Atom: H
Atom: H2, Residue: MOL, Residue Index: 0 Atom: H
Atom: H3, Residue: MOL, Residue Index: 0 Atom: H
Atom: H4, Residue: MOL, Residue Index: 0 Atom: H
Atom: H5, Residue: MOL, Residue Index: 0 Atom: H
Atom: H6, Residue: MOL, Residue Index: 0 Atom: H
Atom: H7, Residue: MOL, Residue Index: 0 Atom: H
Atom: H8, Residue: MOL, Residue Index: 0 Atom: H
Atom: H9, Residue: MOL, Residue Index: 0 Atom: H
Atom: H10, Residue: MOL, Residue Index: 0 Atom: H
Atom: C1, Residue: MOL, Residue Index: 0 Atom: C
Atom: C2, Residue: MOL, Residue Index: 0 Atom: C
Atom: C3, Residue: MOL, Residue Index: 0 Atom: C
Atom: C4, Residue: MOL, Residue Index: 0 Atom: C
Atom: C5, Residue: MOL, Residue Index: 0 Atom: C
Atom: C6, Residue: MOL, Residue Index: 0 Atom: C
Atom: C7, R

In [38]:
pdb.topology

<Topology; 1 chains, 2 residues, 43 atoms, 0 bonds>

In [24]:
force = system.getForces()[0]
force.setUseDispersionCorrection(False)

AttributeError: 'CustomNonbondedForce' object has no attribute 'setUseDispersionCorrection'

In [25]:
pdb.positions

Quantity(value=[Vec3(x=0.2013, y=-0.18580000000000002, z=0.5), Vec3(x=-0.046400000000000004, y=0.12290000000000001, z=0.9998), Vec3(x=-0.041800000000000004, y=0.2318, z=0.9999000000000001), Vec3(x=0.46870000000000006, y=0.1043, z=0.3285), Vec3(x=0.3159, y=0.21450000000000002, z=0.8331), Vec3(x=0.3234, y=0.33020000000000005, z=0.8338)], unit=nanometer)

In [103]:
# Load a force field
forcefield = app.ForceField("custom_forcefield.xml")

# Create a System based on the force field
system = forcefield.createSystem(pdb.topology)

# Access the NonbondedForce (assuming it's used in the force field)
for force_index in range(system.getNumForces()):
    force = system.getForce(force_index)
    if isinstance(force, mm.NonbondedForce):
        custom_force = mm.CustomNonbondedForce("charge*q")
        custom_force.addPerParticleParameter("charge")
        
        # Copy existing particles and parameters
        for i in range(force.getNumParticles()):
            params = force.getParticleParameters(i)
            custom_force.addParticle(params)
        
        # Replace the existing NonbondedForce with the CustomNonbondedForce
        system.removeForce(force_index)
        system.addForce(custom_force)
        break

In [105]:
isinstance(force, mm.NonbondedForce)

False

In [29]:


# Load PDB file
pdb = app.PDBFile("MIL120_loading.pdb")

# Get the topology from the PDB file
topology = pdb.topology

# Get the number of atoms
num_atoms = topology.getNumAtoms()

print(f"Number of atoms in the PDB file: {num_atoms}")


Number of atoms in the PDB file: 6


In [32]:
for i in pdb.topology.atoms():
    print(i)

<Atom 0 (Al) of chain 0 residue 0 (MOL)>
<Atom 1 (H) of chain 0 residue 0 (MOL)>
<Atom 2 (C) of chain 0 residue 0 (MOL)>
<Atom 3 (O) of chain 0 residue 0 (MOL)>
<Atom 4 (O) of chain 0 residue 1 (GAS)>
<Atom 5 (C) of chain 0 residue 1 (GAS)>


In [104]:
custom_force.setParticleParameters(0, [1.0])

NameError: name 'custom_force' is not defined

In [100]:
system.setParticleMass

<bound method System.setParticleMass of <openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x7fea49aecba0> >>

In [101]:
system.setParticleCharge

AttributeError: 'System' object has no attribute 'setParticleCharge'

In [86]:
for index, atom in enumerate(pdb.topology.atoms()):
    atom.charge=0
    print(atom, index)
    #ff.registerAtom(index, atom.name, atom.residue.name, atom.element.symbol, charge)

AttributeError: 'PDBFile' object has no attribute 'atoms'

In [109]:
for i in pdb.topology.atoms():
    print()

AttributeError: 'Atom' object has no attribute 'charges'

In [93]:
new_topology = app.Topology()
for chain in pdb.topology.chains():
    new_chain = new_topology.addChain()
    for residue in chain.residues():
        new_residue = new_topology.addResidue(residue.name, new_chain)
        for atom in residue.atoms():
            new_atom = new_topology.addAtom(atom.name, atom.element, new_residue)
            new_atom.charge = 1
            print("one times")


one times
one times
one times
one times
one times
one times


In [ ]:
def runMD_gas_NVT(topfile, pdbfile, trajfile, length):
    try:
        os.remove("GAS.top")
    except:
        pass
    top_prm = parmed.load_file(topfile)
    top_GAS = top_prm 
    top_GAS.save("GAS.top")
    pdb = app.PDBFile(pdbfile)
    top = app.GromacsTopFile("GAS.top")
    top.topology.setPeriodicBoxVectors(pdb.topology.getPeriodicBoxVectors())
    system = top.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds, hydrogenMass=3*unit.dalton)
    for force in system.getForces():
        if isinstance(force, mm.NonbondedForce):
            force.setUseDispersionCorrection(False)
    integ = mm.LangevinIntegrator(SET_temperature*unit.kelvin, 1/unit.picosecond, 2.5*unit.femtosecond)
    simulation = app.Simulation(top.topology, system, integ)
    simulation.reporters.append(app.StateDataReporter(sys.stdout, gap_step, time=True, potentialEnergy=True, temperature=True, density=True, speed=True, remainingTime=True, totalSteps=int(length) * gap_step))
    simulation.reporters.append(app.DCDReporter(trajfile, gap_step))
    simulation.context.setPositions(pdb.getPositions())
    simulation.minimizeEnergy(maxIterations=200)
    simulation.step(int(length) * gap_step)
    os.remove("GAS.top")

In [44]:

# Create a System and a NonbondedForce
system = mm.System()
nonbonded_force = mm.NonbondedForce()

# You can set the Nonbonded method and cutoff here if needed
# nonbonded_force.setNonbondedMethod(openmm.NonbondedForce.CutoffNonPeriodic)
# nonbonded_force.setCutoffDistance(1.0)

# Add particles to the system with their parameters one by one
masses = [1.0, 12.0, 1.0, 16.0, 16.0, 12.0]  # Particle masses in atomic mass units (amu)

# Iterate through your atom data and add them to the NonbondedForce
for atom in atom_data:
    particle_index = system.addParticle(elem_masses[atom['label']])
    # Charge is set to 0.0 for LJ-only interactions
    nonbonded_force.addParticle(atom['charge'], atom['epsilon'], atom['sigma'])  # charge, sigma (nm), epsilon (kJ/mol)

# Add the NonbondedForce to the system
system.addForce(nonbonded_force)
system.getForces()

[<openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7f11bcff5770> >]

In [45]:
import xml.etree.ElementTree as ET

from simtk.openmm import app, NonbondedForce

# Create a NonbondedForce object (replace this with your actual NonbondedForce object)
#nonbonded_force = NonbondedForce()

# Create an XML ElementTree
root = ET.Element("NonbondedForce")

# Iterate through particles in the NonbondedForce and add their parameters to the XML
for i in range(nonbonded_force.getNumParticles()):
    charge, sigma, epsilon = nonbonded_force.getParticleParameters(i)
    particle_elem = ET.SubElement(root, "Particle")
    particle_elem.set("index", str(i))
    particle_elem.set("charge", str(charge))
    particle_elem.set("sigma", str(sigma))
    particle_elem.set("epsilon", str(epsilon))

# Create an ElementTree and write it to an XML file
tree = ET.ElementTree(root)
tree.write("nonbonded_force.xml")

print("NonbondedForce parameters saved to nonbonded_force.xml")


NonbondedForce parameters saved to nonbonded_force.xml


In [56]:
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom

# Create the root element
root = ET.Element("ForceField")

# Create the LennardJonesForce element
lj_force = ET.SubElement(root, "LennardJonesForce")
lj_force.set("lj14scale", "0.50000")

# Define atom data
atom_data = [
    {"epsilon": "0.359824", "sigma": "0.3399670000000001", "type": "ca"},
    {"epsilon": "0.06276", "sigma": "0.25996400000000003", "type": "ha"},
    {"epsilon": "0.45773", "sigma": "0.3399670000000001", "type": "c3"},
    {"epsilon": "0.0656888", "sigma": "0.247135", "type": "h1"},
    {"epsilon": "0.880314", "sigma": "0.306647", "type": "oh"},
    {"epsilon": "0.0", "sigma": "0.0", "type": "ho"},
]

# Add Atom elements for each atom
for atom_info in atom_data:
    atom = ET.SubElement(lj_force, "Atom")
    atom.set("epsilon", atom_info["epsilon"])
    atom.set("sigma", atom_info["sigma"])
    atom.set("type", atom_info["type"])

# Create an ElementTree
tree = ET.ElementTree(root)

# Convert the ElementTree to a formatted string with line breaks
xml_string = minidom.parseString(ET.tostring(root)).toprettyxml(indent="    ")

# Remove the XML declaration line
xml_lines = xml_string.split("\n")[1:]

# Write the formatted XML string to a file
with open("custom_forcefield.xml", "w") as xml_file:
    xml_file.write("\n".join(xml_lines))

print("XML file created: custom_forcefield.xml")


XML file created: custom_forcefield.xml


In [50]:
xmlio = XMLIO()
xmlio.loadXML("openmm_forcefield.xml")

In [81]:
from ase.io import read, write
atoms = read("/home/yutao/project/MIL-120/adsoption/site1/co2_1.cif")
print(atoms.get_chemical_symbols(), atoms.get_positions())

['O', 'C', 'O'] [[ 4.13272685 -2.97431954  8.34502069]
 [ 3.23378102  3.30204889  8.3380211 ]
 [ 3.15856622  2.14451582  8.3310215 ]]


In [12]:
top = app.GromacsTopFile("Lig_500.top")
top.topology.setPeriodicBoxVectors(pdb.topology.getPeriodicBoxVectors())

FileNotFoundError: [Errno 2] No such file or directory: 'Lig_500.top'

In [13]:
ff.createSystem??

Signature:
ff.createSystem(
    topology,
    nonbondedMethod=NoCutoff,
    nonbondedCutoff=Quantity(value=1.0, unit=nanometer),
    constraints=None,
    rigidWater=None,
    removeCMMotion=True,
    hydrogenMass=None,
    residueTemplates={},
    ignoreExternalBonds=False,
    switchDistance=None,
    flexibleConstraints=False,
    drudeMass=Quantity(value=0.4, unit=dalton),
    **args,
)
Source:   
    def createSystem(self, topology, nonbondedMethod=NoCutoff, nonbondedCutoff=1.0*unit.nanometer,
                     constraints=None, rigidWater=None, removeCMMotion=True, hydrogenMass=None, residueTemplates=dict(),
                     ignoreExternalBonds=False, switchDistance=None, flexibleConstraints=False, drudeMass=0.4*unit.amu, **args):
        """Construct an OpenMM System representing a Topology with this force field.

        Parameters
        ----------
        topology : Topology
            The Topology for which to create a System
        nonbondedMethod : object=NoCutof

This part use a structure from UFF optimization and run geometry optimization 

In [80]:
def runMD(topfile, pdbfile, trajfile, length):
    try:
        os.remove("Lig_500.top")
    except:
        pass
    top_prm = parmed.load_file(topfile)
    top_500 = top_prm * 500
    top_500.save("Lig_500.top")
    pdb = app.PDBFile(pdbfile)
    top = app.GromacsTopFile("Lig_500.top")
    top.topology.setPeriodicBoxVectors(pdb.topology.getPeriodicBoxVectors())
    system = top.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds, hydrogenMass=3*unit.dalton)
    for force in system.getForces():
        if isinstance(force, mm.NonbondedForce):
            force.setUseDispersionCorrection(False)
    integ = mm.LangevinIntegrator(200 *unit.kelvin, 1/unit.picosecond, 2.5*unit.femtosecond)
    simulation = app.Simulation(top.topology, system, integ)
    simulation.reporters.append(app.StateDataReporter(sys.stdout, 400, time=True, potentialEnergy=True, temperature=True, density=True, speed=True, remainingTime=True, totalSteps=int(length) * 400))
    simulation.reporters.append(app.DCDReporter(trajfile, 400))
    simulation.context.setPositions(pdb.getPositions())
    simulation.minimizeEnergy(maxIterations=200)
    simulation.step(int(length) * 400)
    os.remove("Lig_500.top")

# runMD("Lig.top", "init.pdb", "traj.dcd", 100)

In [19]:
system.getForces()

[<openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7f24735d0540> >,
 <openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x7f24735d0210> >]

In [14]:
top = app.GromacsTopFile("MIL120.top")
top.topology.setPeriodicBoxVectors(pdb.topology.getPeriodicBoxVectors())
system = top.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds, hydrogenMass=3*unit.dalton)

ValueError: Found [ atoms ] section before [ moleculetype ]

In [ ]:
os.system("cp Lig.top loop-0.top")
for nloop in range(1, 51):
    # sample
    print("SAMPLE")
    runMD(f"loop-{nloop-1}.top", "init.pdb", f"loop-{nloop}.dcd", length=70)
    print("RERUN")
    ener = rerun_energy("init.pdb", f"loop-{nloop}.dcd", f"loop-{nloop-1}.top", removeLJ=False, skip=20)
    ener_no_lj = rerun_energy("init.pdb", f"loop-{nloop}.dcd", f"loop-{nloop-1}.top", skip=20)
    print("ESTIMATOR")
    traj = md.load(f"loop-{nloop}.dcd", top="init.pdb")[20:]
    estimator = ReweightEstimator(ener, base_energies=ener_no_lj, volume=traj.unitcell_volumes)

    print("CALC DENSE")
    density = md.density(traj) * 0.001

    # get loss & grad
    def loss(paramset):
        lj_jax = rerun_dmff_lennard_jones(paramset, "init.pdb", f"loop-{nloop}.dcd", lj_force, skip=20)
        weight = estimator.estimate_weight(lj_jax)
        dens = weight * density
        dens = dens.mean()
        return jnp.power(dens - 0.85, 2)
    
    v_and_g = jax.value_and_grad(loss, 0)
    v, g = v_and_g(paramset)
    print("Loss:", v)
    # update parameters
    updates, opt_state = optimizer.update(g, opt_state)
    paramset = optax.apply_updates(paramset, updates)
    paramset = jax.tree_map(lambda x: jnp.clip(x, 0.0, 1e8), paramset)
    
    # upate ffinfo
    lj_gen.overwrite(paramset)
    prmop.overwriteLennardJones(prm_top, lj_gen.ffinfo)
    prm_top.save(f"loop-{nloop}.top")

In [51]:
ffinfo = xmlio.parseXML()

paramset = ParamSet()
lj_gen = LennardJonesGenerator(ffinfo, paramset)

KeyError: 'LennardJonesForce'

In [49]:
ffinfo

{'Operators': {},
 'AtomTypes': [{'name': 'Al', 'class': 'Al', 'element': 'Al'},
  {'name': 'C', 'class': 'C', 'element': 'C'},
  {'name': 'H', 'class': 'H', 'element': 'H'},
  {'name': 'O', 'class': 'O', 'element': 'O'},
  {'name': 'O_co2', 'class': 'O_co2', 'element': 'O'},
  {'name': 'C_co2', 'class': 'C_co2', 'element': 'C'}],
 'Residues': [],
 'Forces': {'NonbondedForce': {'meta': {'coulomb14scale': '0.833333',
    'lj14scale': '0.5'},
   'node': [{'name': 'Atom',
     'attrib': {'type': 'Al',
      'charge': '0.0',
      'sigma': '4.0082',
      'epsilon': '254.152'}},
    {'name': 'Atom',
     'attrib': {'type': 'C',
      'charge': '0.0',
      'sigma': '3.4309',
      'epsilon': '52.8435'}},
    {'name': 'Atom',
     'attrib': {'type': 'H',
      'charge': '0.0',
      'sigma': '2.5711',
      'epsilon': '22.1439'}},
    {'name': 'Atom',
     'attrib': {'type': 'O',
      'charge': '0.0',
      'sigma': '3.1181',
      'epsilon': '30.1963'}},
    {'name': 'Atom',
     'attrib'

In [37]:
import openmm.app as app

# Create a ForceField object
forcefield = app.ForceField()

# Add a NonbondedForce to the ForceField (similar to what you've defined)
# Define your atom data and LJ parameters
atom_data = [
    {'label': 'Al', 'charge': 0.0, 'epsilon': 2.1019, 'sigma': 0.40082},
    {'label': 'C', 'charge': 0.0, 'epsilon': 0.4400, 'sigma': 0.34309},
    {'label': 'H', 'charge': 0.0, 'epsilon': 0.1841, 'sigma': 0.25711},
    {'label': 'O', 'charge': 0.0, 'epsilon': 0.2513, 'sigma': 0.31181}
]

for atom in atom_data:
    forcefield.registerParticle(
        atom['label'],
        atom['epsilon'],
        atom['sigma']
    )

# Save the force field to an XML file
forcefield.to_xml('custom_forcefield.xml')


AttributeError: 'ForceField' object has no attribute 'registerParticle'

In [15]:
xmlio = XMLIO()
xmlio.loadXML("custom_forcefield.xml")
ffinfo = xmlio.parseXML()

In [18]:
ffinfo['Forces']

{'LennardJonesForce': {'meta': {'lj14scale': '0.50000'},
  'node': [{'name': 'Atom',
    'attrib': {'epsilon': '2.1019', 'sigma': '0.40082', 'type': 'Al'}},
   {'name': 'Atom',
    'attrib': {'epsilon': '0.44', 'sigma': '0.34309', 'type': 'C'}},
   {'name': 'Atom',
    'attrib': {'epsilon': '0.1841', 'sigma': '0.25711', 'type': 'H'}},
   {'name': 'Atom',
    'attrib': {'epsilon': '0.2513', 'sigma': '0.31181', 'type': 'O'}},
   {'name': 'Atom',
    'attrib': {'epsilon': '0.6577', 'sigma': '0.305', 'type': 'O_co2'}},
   {'name': 'Atom',
    'attrib': {'epsilon': '0.2249', 'sigma': '0.28', 'type': 'C_co2'}}]}}

In [ ]:
import openmm.app as app
import openmm.unit as unit
from simtk.openmm import System, NonbondedForce, LangevinIntegrator, Platform, Context

# Create an OpenMM System
system = System()

# Define LJ parameters for your elements
lj_parameters = {
    'Al': (2.1019, 0.40082),
    'C': (0.4400, 0.34309),
    'H': (0.1841, 0.25711),
    'O': (0.2513, 0.31181)
}

# Add particles to the system with masses, LJ parameters, and initial positions
for element, params, initial_position in [
    ('Al', lj_parameters['Al'], (0.0, 0.0, 0.0)),
    ('C', lj_parameters['C'], (1.0, 0.0, 0.0)),
    ('H', lj_parameters['H'], (0.0, 1.0, 0.0)),
    ('O', lj_parameters['O'], (0.0, 0.0, 1.0))
]:
    mass = 1.0  # Specify the mass (you can change it)
    charge = 0.0  # Set to 0 for LJ-only interactions
    epsilon, sigma = params
    particle_index = system.addParticle(mass)
    system.setParticleParameters(particle_index, charge, sigma, epsilon)
    system.setParticlePosition(particle_index, initial_position)

# Create a NonbondedForce for LJ interactions
nonbonded_force = NonbondedForce()
nonbonded_force.setNonbondedMethod(NonbondedForce.CutoffNonPeriodic)
nonbonded_force.setCutoffDistance(1.0)  # Set the nonbonded cutoff distance

# Add the NonbondedForce to the system
system.addForce(nonbonded_force)

# Set up the integrator
temperature = 300.0 * unit.kelvin
friction = 1.0 / unit.picosecond
step_size = 2.0 * unit.femtosecond
integrator = LangevinIntegrator(temperature, friction, step_size)

# Create a simulation context
platform = Platform.getPlatformByName('Reference')  # Change this to your desired platform
context = Context(system, integrator, platform)

# Get and print the positions from the context
positions = context.getState(getPositions=True).getPositions()
print(positions)


In [3]:
from simtk.openmm.app import ForceField, Element

# Conversion factor from K to kJ/mol
k_to_kJ = 0.008314462618

# Create an OpenMM ForceField object
ff = ForceField()

# Define LJ parameters for individual atom types with accurate unit conversion
ff.registerAtomType(
    sigma=2.80,  # Convert units from Ångstroms to nm
    epsilon=2.7 * k_to_kJ,  # Convert units from K to kJ/mol
    element=Element.getBySymbol('C'),
    id='C_co2'
)

ff.registerAtomType(
    sigma=3.05,  # Convert units from Ångstroms to nm
    epsilon=6.6 * k_to_kJ,  # Convert units from K to kJ/mol
    element=Element.getBySymbol('O'),
    id='O_co2'
)

ff.registerAtomType(
    sigma=3.4309,  # Convert units from Ångstroms to nm
    epsilon=2.5 * k_to_kJ,  # Convert units from K to kJ/mol
    element=Element.getBySymbol('C'),
    id='C'
)

ff.registerAtomType(
    sigma=2.5711,  # Convert units from Ångstroms to nm
    epsilon=0.9 * k_to_kJ,  # Convert units from K to kJ/mol
    element=Element.getBySymbol('H'),
    id='H'
)

ff.registerAtomType(
    sigma=3.1181,  # Convert units from Ångstroms to nm
    epsilon=1.2 * k_to_kJ,  # Convert units from K to kJ/mol
    element=Element.getBySymbol('O'),
    id='O'
)

ff.registerAtomType(
    sigma=4.0082,  # Convert units from Ångstroms to nm
    epsilon=26.7 * k_to_kJ,  # Convert units from K to kJ/mol
    element=Element.getBySymbol('Al'),
    id='Al'
)

# Generate the XML representation of the force field
xml_string = ff._generateXML()

# Save the XML string to a file for use in simulations
'''
with open('openmm_forcefield.xml', 'w') as f:
    f.write(xml_string)
'''

TypeError: ForceField.registerAtomType() got an unexpected keyword argument 'sigma'

In [4]:
from simtk.openmm.app import ForceField, Element

# Create an OpenMM ForceField object
ff = ForceField()

# Define atom types without LJ parameters here

# Generate the XML representation of the force field
xml_string = ff._generateXML()

# Now, add the LJ parameters to the generated XML string
lj_parameters = """
<NonbondedForce coulomb14scale="0.833333" lj14scale="0.5">
    <Atom type="Al" charge="0.0" sigma="4.0082" epsilon="26.7"/>
    <Atom type="C" charge="0.0" sigma="3.4309" epsilon="52.8435"/>
    <Atom type="H" charge="0.0" sigma="2.5711" epsilon="22.1439"/>
    <Atom type="O" charge="0.0" sigma="3.1181" epsilon="30.1963"/>
    <Atom type="O_co2" charge="0.0" sigma="3.05" epsilon="79.0"/>
    <Atom type="C_co2" charge="0.0" sigma="2.80" epsilon="27.0"/>
</NonbondedForce>
"""

# Insert the LJ parameters into the XML string
xml_string = xml_string.replace("</ForceField>", lj_parameters + "\n</ForceField>")

# Save the XML string to a file for use in simulations
'''
with open('openmm_forcefield.xml', 'w') as f:
    f.write(xml_string)
'''

AttributeError: 'ForceField' object has no attribute '_generateXML'

In [20]:
from CifFile import ReadCif

# Load the CIF file
cif_file = "/home/yutao/softwares/RASPA2/../MIL-120/MIL-120.cif"

# Read the CIF file
cif_data = ReadCif(cif_file)

# Extract charge information
charges = {}

# Loop through the CIF data blocks
for block in cif_data:

    if '_atom_site_label' in block and '_atom_site_charge' in block:
        labels = block['_atom_site_label']
        charge_values = block['_atom_site_charge']

        for label, charge in zip(labels, charge_values):
            charge = float(charge)
            if label in charges:
                charges[label].append(charge)
            else:
                charges[label] = [charge]

# Now you have a dictionary with atom labels as keys and lists of charges as values
print(charges)


{'Al': [1.65857, 1.65856, 1.64626, 1.64749], 'H': [0.41785, 0.41373, 0.41657, 0.41461, 0.41773, 0.41364, 0.41658, 0.41466, 0.12728, 0.12726], 'C': [0.62462, -0.02738, 0.62697, -0.0317, 0.62454, -0.02728, 0.62705, -0.03187, -0.11722, -0.11724], 'O': [-0.9263, -0.97561, -0.60514, -0.57838, -0.92481, -0.97588, -0.60682, -0.57781, -0.92626, -0.97553, -0.60522, -0.57825, -0.92474, -0.97585, -0.60674, -0.57792]}


In [23]:
from CifFile import ReadCif

# Load the CIF file
cif_file = "/home/yutao/softwares/RASPA2/../MIL-120/MIL-120.cif"

# Read the CIF file
cif_data = ReadCif(cif_file)

# Extract information for each atom
atom_data = []

# Loop through the CIF data blocks
for block in cif_data:
    if '_atom_site_label' in block and '_atom_site_charge' in block and '_atom_site_fract_x' in block and '_atom_site_fract_y' in block and '_atom_site_fract_z' in block:
        labels = block['_atom_site_label']
        charge_values = block['_atom_site_charge']
        x_positions = block['_atom_site_fract_x']
        y_positions = block['_atom_site_fract_y']
        z_positions = block['_atom_site_fract_z']

        for label, charge, x, y, z in zip(labels, charge_values, x_positions, y_positions, z_positions):
            atom_data.append({
                'label': label,
                'charge': float(charge),
                'position': (float(x), float(y), float(z))
            })

# Now you have a list of dictionaries, each containing label, charge, and position
print(atom_data)



[{'label': 'Al', 'charge': 1.65857, 'position': (0.29166, 0.29186, 0.58401)}, {'label': 'Al', 'charge': 1.65856, 'position': (0.70833, 0.70814, 0.416)}, {'label': 'Al', 'charge': 1.64626, 'position': (-0.0, 0.5, 0.5)}, {'label': 'Al', 'charge': 1.64749, 'position': (0.5, -0.0, 0.5)}, {'label': 'H', 'charge': 0.41785, 'position': (0.01775, 0.34267, 0.67087)}, {'label': 'H', 'charge': 0.41373, 'position': (0.14093, 0.93835, 0.4632)}, {'label': 'H', 'charge': 0.41657, 'position': (0.65156, 0.32865, 0.67108)}, {'label': 'H', 'charge': 0.41461, 'position': (0.32343, 0.52517, 0.46387)}, {'label': 'H', 'charge': 0.41773, 'position': (0.98225, 0.65733, 0.32913)}, {'label': 'H', 'charge': 0.41364, 'position': (0.85906, 0.06165, 0.5368)}, {'label': 'H', 'charge': 0.41658, 'position': (0.34843, 0.67135, 0.32893)}, {'label': 'H', 'charge': 0.41466, 'position': (0.67656, 0.47484, 0.53613)}, {'label': 'H', 'charge': 0.12728, 'position': (0.54939, 0.33475, 0.00012)}, {'label': 'H', 'charge': 0.12726,

In [5]:
from simtk.openmm import app
from simtk.openmm.app.internal import createSystem
from simtk.unit import nanometer, kilojoule_per_mole, element
from simtk.openmm import NonbondedForce

# Create an OpenMM System
system = createSystem(None, nonbondedMethod=app.NoCutoff)

# Create a NonbondedForce
nonbonded = NonbondedForce()

# Define LJ parameters for individual atom types
lj_parameters = [
    ("Al", 4.0082 * nanometer, 254.152 * kilojoule_per_mole),
    ("C", 3.4309 * nanometer, 52.8435 * kilojoule_per_mole),
    ("H", 2.5711 * nanometer, 22.1439 * kilojoule_per_mole),
    ("O", 3.1181 * nanometer, 30.1963 * kilojoule_per_mole),
    ("O_co2", 3.05 * nanometer, 79.0 * kilojoule_per_mole),
    ("C_co2", 2.80 * nanometer, 27.0 * kilojoule_per_mole),
]

for atom_type, sigma, epsilon in lj_parameters:
    nonbonded.addParticle(0.0, sigma, epsilon)

# Add NonbondedForce to the System
system.addForce(nonbonded)

# Create an OpenMM XML force field file
#app.internal.writeForceFieldFile(system, 'openmm_forcefield.xml')


ModuleNotFoundError: No module named 'simtk.openmm.app.internal'

In [6]:
from simtk.openmm.app import Element, ForceField

# Create an OpenMM ForceField object
ff = ForceField()

# Define LJ parameters for individual atom types with accurate unit conversion
ff.registerAtomType(
    element=Element.getBySymbol('Al'),
    sigma=4.0082,  # The units are in Ångstroms
    epsilon=254.152  # The units are in Kelvin (K)
)

ff.registerAtomType(
    element=Element.getBySymbol('C'),
    sigma=3.4309,  # The units are in Ångstroms
    epsilon=52.8435  # The units are in Kelvin (K)
)

ff.registerAtomType(
    element=Element.getBySymbol('H'),
    sigma=2.5711,  # The units are in Ångstroms
    epsilon=22.1439  # The units are in Kelvin (K)
)

ff.registerAtomType(
    element=Element.getBySymbol('O'),
    sigma=3.1181,  # The units are in Ångstroms
    epsilon=30.1963  # The units are in Kelvin (K)
)

ff.registerAtomType(
    element=Element.getBySymbol('O'),
    sigma=3.05,  # The units are in Ångstroms
    epsilon=79.0  # The units are in Kelvin (K)
)

ff.registerAtomType(
    element=Element.getBySymbol('C'),
    sigma=2.80,  # The units are in Ångstroms
    epsilon=27.0  # The units are in Kelvin (K)
)

# Generate the XML representation of the force field
xml_string = ff._generateXML()

# Save the XML string to a file for use in simulations
with open('openmm_forcefield.xml', 'w') as f:
    f.write(xml_string)


TypeError: ForceField.registerAtomType() got an unexpected keyword argument 'element'

In [7]:
ForceField.registerAtomType??

Signature: ForceField.registerAtomType(self, parameters)
Source:   
    def registerAtomType(self, parameters):
        """Register a new atom type."""
        name = parameters['name']
        if name in self._atomTypes:
            raise ValueError('Found multiple definitions for atom type: '+name)
        atomClass = parameters['class']
        mass = _convertParameterToNumber(parameters['mass'])
        element = None
        if 'element' in parameters:
            element = parameters['element']
            if not isinstance(element, elem.Element):
                element = elem.get_by_symbol(element)
        self._atomTypes[name] = ForceField._AtomType(name, atomClass, mass, element)
        if atomClass in self._atomClasses:
            typeSet = self._atomClasses[atomClass]
        else:
            typeSet = set()
            self._atomClasses[atomClass] = typeSet
        typeSet.add(name)
        self._atomClasses[''].add(name)
File:      ~/anaconda3/envs/dmff/lib/python3.11/

In [8]:
from simtk.openmm.app import ForceField

# Create an OpenMM ForceField object
ff = ForceField()

# Generate the XML representation of the force field
xml_string = """
<ForceField>
    <AtomTypes>
        <Type name="Al" class="Al" element="Al"/>
        <Type name="C" class="C" element="C"/>
        <Type name="H" class="H" element="H"/>
        <Type name="O" class="O" element="O"/>
        <Type name="O_co2" class="O_co2" element="O"/>
        <Type name="C_co2" class="C_co2" element="C"/>
    </AtomTypes>
    <NonbondedForce coulomb14scale="0.833333" lj14scale="0.5">
        <Atom type="Al" charge="0.0" sigma="4.0082" epsilon="254.152"/>
        <Atom type="C" charge="0.0" sigma="3.4309" epsilon="52.8435"/>
        <Atom type="H" charge="0.0" sigma="2.5711" epsilon="22.1439"/>
        <Atom type="O" charge="0.0" sigma="3.1181" epsilon="30.1963"/>
        <Atom type="O_co2" charge="0.0" sigma="3.05" epsilon="79.0"/>
        <Atom type="C_co2" charge="0.0" sigma="2.80" epsilon="27.0"/>
    </NonbondedForce>
</ForceField>
"""

# Save the XML string to a file for use in simulations
with open('openmm_forcefield.xml', 'w') as f:
    f.write(xml_string)


In [9]:
xmlio = XMLIO()
xmlio.loadXML("openmm_forcefield.xml")

In [13]:
from simtk.openmm import app, unit

# Create a System
system = openmm.System()

# Define partial charges (in elementary charge units, e)
charges = [0.5, -0.2, 0.3, 0.1, -0.4, 0.0]  # Example charges

# Add particles and set masses
for _ in charges:
    mass = 1.0 * unit.amu  # Mass of 1.0 atomic mass unit (AMU)
    system.addParticle(mass)

# Add Coulombic interactions (Electrostatics)
nonbonded_force = openmm.NonbondedForce()
for i in range(len(charges)):
    nonbonded_force.addParticle(charge=charges[i])

# Set the dielectric constant for Coulombic interactions
nonbonded_force.setReactionFieldDielectric(78.5)

# Add the Coulombic force to the system
system.addForce(nonbonded_force)

# Define the integrator and simulation context
integrator = openmm.LangevinIntegrator(300 * unit.kelvin, 1.0 / unit.picosecond, 0.002 * unit.picoseconds)
simulation = openmm.app.Simulation(pdb.topology, system, integrator)

# Run the simulation
simulation.step(1000)


TypeError: NonbondedForce.addParticle() missing 2 required positional arguments: 'sigma' and 'epsilon'